In [1]:
import os
import numpy as np
import torch
from datetime import datetime
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
import seaborn as sns

import zuko
from zuko.flows import GMM

from pymoo.algorithms.soo.nonconvex.brkga import BRKGA
from pymoo.core.callback import CallbackCollection, Callback
from pymoo.core.problem import ElementwiseProblem, Problem
from pymoo.core.duplicate import ElementwiseDuplicateElimination
from pymoo.problems.functional import FunctionalProblem, func_return_none
from pymoo.optimize import minimize

from jkinpylib.evaluation import evaluate_solutions

from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

from utils.settings import config
from utils.utils import *
from utils.model import *
from utils.robot import Robot

config.py: Using device 'cuda:0'


In [2]:
class ProblemWrapper(ElementwiseProblem):
    
    def __init__(self, 
                 iksols,
                 target_poses,
                 **kwargs):
        self.target_poses = target_poses
        self.all_sols = iksols
        self.all_l2_errs = self.calc_all_l2_errs()
        xl = np.zeros((iksols.shape[0]))
        xu = np.full_like(xl, fill_value=len(iksols)-1)
        super().__init__(n_var=len(xl),
                         n_obj=1,
                         n_ieq_constr=0,
                         n_eq_constr=0,
                         xl=xl,
                         xu=xu,
                         **kwargs)
    
    def calc_all_l2_errs(self):
        all_l2_errs = np.ones((self.all_sols.shape[0], self.all_sols.shape[1]))
        for i in range(self.all_sols.shape[1]):
            qtraj = self.all_sols[:, i, :]
            l2_errs, _, _, _ = evaluate_solutions(
                ik_solver.robot, 
                torch.tensor(self.target_poses), 
                torch.tensor(qtraj))
            all_l2_errs[:, i] = l2_errs
        return all_l2_errs
    
    def ang_jump(self, qs):
        errs = calc_ang_errs(qs)
        return errs.mean()
    
    def pheno(self, design):
        return np.floor(design).astype(int)
        
    def iksols(self, pheno):
        qs = np.zeros((self.all_sols.shape[0], self.all_sols.shape[2]))
        for ri, i in enumerate(pheno):
            qs[ri] = self.all_sols[ri, i]
        return qs

    def l2_errs(self, pheno):
        l2_errs = 0
        for ri, i in enumerate(pheno):
            l2_errs += self.all_l2_errs[ri, i]
        return l2_errs / len(pheno)

    def _evaluate(self, design, out, *args, **kwargs):
        pheno = self.pheno(design)
        iksols = self.iksols(pheno=pheno)
        ang_errs = self.ang_jump(qs=iksols)
        l2_errs = self.l2_errs(pheno=pheno)
        out["F"] = ang_errs
        out["ang_errs"] = ang_errs
        out["l2_errs"] = l2_errs
        out["pheno"] = pheno
        out["hash"] = hash(str(pheno))

class MyElementwiseDuplicateElimination(ElementwiseDuplicateElimination):

    def is_equal(self, a, b):
        return a.get("hash") == b.get("hash")

In [3]:
def get_target_poses(robot): 
    traj_dir = sample_ee_traj(robot=robot, load_time='')
    ee_traj = load_numpy(file_path=traj_dir + 'ee_traj.npy')
    quaternions = np.zeros((len(ee_traj), 4))
    quaternions[:, 0] = np.random.randn() * 2e-2 + 1
    target_poses = np.column_stack((ee_traj, quaternions))
    return traj_dir, target_poses

In [8]:
file_names = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
exp_5_fig_dir = config.traj_dir + f'figs/exp_5_{datetime.now().strftime("%m%d%H%M")}/'
if not os.path.exists(path=exp_5_fig_dir):
    os.makedirs(exp_5_fig_dir)
set_seed()
num_trails = 3
num_generation = 100
num_ikflow_trails = num_generation
num_solutions = 500
# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot
panda = Robot(verbose=False)

set_seed() - random int:  44
ndim_tot=7
dim_cond=8
WorldModel::LoadRobot: /tmp/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
URDFParser: Done loading robot file /tmp/panda_arm_hand_formatted_link_filepaths_absolute.urdf


In [13]:
for n_trail in trange(num_trails):
    traj_dir, target_poses = get_target_poses(robot=panda)
    exp_dir = traj_dir + f'exp_5_single_obj/'
    data = {fn: [] for fn in file_names}
    
    # -> unrefined solutions
    l2_errs = np.zeros((num_ikflow_trails,))
    ang_errs = np.zeros((num_ikflow_trails,))
    for ikflow_i in range(num_ikflow_trails):
        solutions, l2_err, _, _, _, _ = ik_solver.solve_n_poses(
            target_poses, 
            refine_solutions=False, 
            return_detailed=True)

        iksols = solutions.detach().cpu().numpy()
        df = qtraj_evaluation(robot=panda, qs=iksols, l2_errs=l2_err)
        l2_errs[ikflow_i], ang_errs[ikflow_i] = df.mean().values
    data['ikflow_l2'] = l2_errs
    data['ikflow_ang'] = ang_errs    
    print(np.mean(l2_errs))
    print(np.mean(ang_errs))
    
    # # Generate iksols
    # iksols = np.zeros((len(target_poses), num_solutions, panda.dof))

    # for i, target_pose in enumerate(target_poses):
    #     solutions = ik_solver.solve(
    #         target_pose, 
    #         num_solutions, 
    #         refine_solutions=False, 
    #         return_detailed=False)
    #     iksols[i] = solutions.detach().cpu().numpy()
        
    # problem = ProblemWrapper(
    #     iksols=iksols, 
    #     target_poses=target_poses)

    # algorithm = BRKGA(
    #     n_elites=200,
    #     n_offsprings=700,
    #     n_mutants=300,
    #     bias=0.7)

    # res = minimize(
    #     problem,
    #     algorithm,
    #     ("n_gen", num_generation),
    #     seed=1,
    #     verbose=False,
    #     save_history=True,
    #     eliminate_duplicates=MyElementwiseDuplicateElimination())
    
    # hist = res.history

    
    # for algo in hist:
    #     data['n_evals'].append(algo.evaluator.n_eval)
        
    #     opt = algo.opt
    #     data['l2_errs_min'].append(opt.get("l2_errs").min())
    #     data['l2_errs_avg'].append(algo.pop.get("l2_errs").mean())
        
    #     data['ang_errs_min'].append(opt.get("ang_errs").min())
    #     data['ang_errs_avg'].append(algo.pop.get("ang_errs").mean())
        
    #     data['F_min'].append(algo.pop.get("F").min())
    #     data['F_avg'].append(algo.pop.get("F").mean())
        
    # for key, val in data.items():
    #     # replace this line by `hist_cv` if you like to analyze the least feasible optimal solution and not the population
    #     save_numpy(file_path=exp_dir + f'{key}.npy', arr=np.array(val))

  0%|          | 0/3 [00:00<?, ?it/s]

mkdir /home/luca/ikpflow/data/panda/trajectory/0710020345
/home/luca/ikpflow/data/panda/trajectory/0710020345/ load successfully.


 33%|███▎      | 1/3 [00:01<00:03,  1.77s/it]

0.011765626942888236
542.6455801275969
mkdir /home/luca/ikpflow/data/panda/trajectory/0710020347
/home/luca/ikpflow/data/panda/trajectory/0710020347/ load successfully.


 67%|██████▋   | 2/3 [00:03<00:01,  1.66s/it]

0.17057971116658358
517.6033977220058
mkdir /home/luca/ikpflow/data/panda/trajectory/0710020349
/home/luca/ikpflow/data/panda/trajectory/0710020349/ load successfully.


100%|██████████| 3/3 [00:04<00:00,  1.63s/it]

0.023163149303705128
538.8440417764186


In [5]:
def load_traj_dirs(num_trails=num_trails):
    fns = np.array(os.listdir(path=config.traj_dir))
    fns.sort()
    abs_traj_dir = os.path.abspath(config.traj_dir) + '/'
    fns = np.array([abs_traj_dir + fn + '/' for fn in fns])
    return fns[-num_trails-1:-1] 

In [6]:
def load_and_parse(traj_dir):
    exp_dir = traj_dir + 'exp_5_single_obj/'
    file_paths = [exp_dir + fn + '.npy' for fn in file_names]
    
    data = {}
    for i, fn in enumerate(file_names):
        data[fn] = load_numpy(file_path=file_paths[i])
    
    return data

In [7]:
def visualize(data, y_axis, exp_dir, x_axis=None, title="Convergence"):
    plt.figure(figsize=(7, 5))
    fig_dir = exp_dir + 'figs/'
    if not os.path.exists(path=fig_dir):
        os.mkdir(path=fig_dir)
    if x_axis is None:
        plt.plot(data[y_axis], color='black', lw=0.7)
        fig_path = fig_dir + y_axis + '.png'
    else:
        plt.plot(data[x_axis], data[y_axis],  color='black', lw=0.7)
        plt.scatter(data[x_axis], data[y_axis],  facecolor="none", edgecolor='black', marker="p")
        plt.xlabel(x_axis)
        fig_path = fig_dir + x_axis + '_' + y_axis + '.png'
        

    plt.title(title)
    plt.ylabel(y_axis)
    
    plt.savefig(fig_path)

In [ ]:
def combine_trail_data(num_trails, num_generation):
    traj_dirs = load_traj_dirs(num_trails)
    name_list = ['ang_errs_avg', 'ang_errs_min', 'F_avg', 'F_min', 'ikflow_ang', 'ikflow_l2', 'l2_errs_avg', 'l2_errs_min', 'n_evals']
    
    trails_data = {name: np.array([]) for name in name_list}
    trails_data['n_trails'] = np.array([])
    trails_data['n_gen'] = np.array([])
    
    n_trail = 0
    for td in tqdm(traj_dirs):
        data = load_and_parse(traj_dir=td)
        trails_data['n_trails'] = np.concatenate((trails_data['n_trails'], np.ones((num_generation))*n_trail))
        trails_data['n_gen'] = np.concatenate((trails_data['n_gen'], list(range(num_generation))))
        for name in name_list:
            trails_data[name] = np.concatenate((trails_data[name], data[name]))
        n_trail += 1
        
    df = pd.DataFrame(data=trails_data)
    
    return df

In [ ]:
def load_and_vis(num_trails, num_generation):
    vis_pair = [
        ('n_evals', 'ang_errs_min', 'Number of Evaluations', 'Joint Jumps (degrees)', 'Joint Jumps of IKB (min)'),
        ('n_evals', 'ang_errs_avg', 'Number of Evaluations', 'Joint Jumps (degrees)', 'Joint Jumps of IKB (avg)'),
        ('n_evals', 'F_min', 'Number of Evaluations', 'F_min', 'F_min'),
        ('n_evals', 'F_avg', 'Number of Evaluations', 'F_avg', 'F_avg'),
        ('n_evals', 'l2_errs_min', 'Number of Evaluations', 'L2 Position Error (m)', 'L2 Position Error of IKB (min)'),
        ('n_evals', 'l2_errs_avg', 'Number of Evaluations', 'L2 Position Error (m)', 'L2 Position Error of IKB (avg)'),
        ('n_gen', 'ikflow_ang', 'Number of Generations', 'Joint Jumps (degrees)', 'Joint Jumps of IKFlow'),
        ('n_gen', 'ikflow_l2', 'Number of Generations', 'L2 Position Error (m)', 'L2 Position Error of IKFlow')
    ]
    
    column_renameing = {
        y: t for _, y, _, _, t in vis_pair
    }
    
    errbar = [('sd'), ('se')]

    trails_data = combine_trail_data(num_trails=num_trails, num_generation=num_generation)
    
    trails_data.rename(columns=column_renameing, inplace=True)
    sns.set(font_scale=1.15)  # crazy big
    for x, y, xa, ya, t in vis_pair:
        for eb in errbar:
            ax = sns.relplot(data=trails_data, x=x, y=t, kind="line").set(title=t, xlabel=xa, ylabel=ya)
            # ax.set_xticklabels(step=1)
            # ax.set_xlabels()
            ax.savefig(f'{exp_5_fig_dir}{t}_{eb}.png')
    

load_and_vis(num_trails=100, num_generation=100)

In [ ]:
# https://stackoverflow.com/questions/13404468/t-test-in-pandas (o)
    # Ttest_indResult(statistic=-370.76733198053574, pvalue=0.0)
    # since the pvalue of this experiment is 0.0 which is lower than the threshold 0.05. That is, our method is significantly improve the smoothness of solutions from diverse learning-based IK method for the path-following tasks. 
# success rates

# standard error (o)
# https://seaborn.pydata.org/tutorial/error_bars.html error_bar methods

# mean best fitness values (o)

In [ ]:
trails_data = combine_trail_data(num_trails=num_trails, num_generation=num_generation)
trails_data

In [ ]:
def load_and_ttest(num_trails=num_trails, num_generation=num_generation):
    trails_data = combine_trail_data(num_trails=num_trails, num_generation=num_generation)
    
    com_pair = [
        ('ang_errs_avg', 'ikflow_ang'),
        ('ang_errs_min', 'ikflow_ang'),
        ('l2_errs_avg', 'ikflow_l2'),
        ('l2_errs_min', 'ikflow_l2'),
    ]
    # trails_data.query('n_evals==100200')['ang_errs_min'].describe()
    for x, y in com_pair:
        stat, pval = ttest_ind(trails_data.query('n_evals==100200')[x], trails_data.query('n_gen==99')[y], equal_var=False)
        print(f"t-test on ours({x}) and ikflow({y}) with stat = {stat}, p-value = {pval}")
    

In [ ]:
success_threshold = 210

In [ ]:
trails_data.query('n_evals==100200 and ang_errs_min <= 280')['ang_errs_min'].describe()

In [ ]:
trails_data.query('ikflow_ang <= 280')['ikflow_ang'].describe()

In [ ]:
load_and_ttest()

In [ ]:
def load_and_dfdescribe(num_trails=num_trails, num_generation=num_generation):
    vis_pair = [
        ('n_evals', 'ang_errs_min'),
        ('n_evals', 'ang_errs_avg'),
        ('n_evals', 'F_min'),
        ('n_evals', 'F_avg'),
        ('n_evals', 'l2_errs_min'),
        ('n_evals', 'l2_errs_avg'),
        ('n_gen', 'ikflow_ang'),
        ('n_gen', 'ikflow_l2')
    ]
    errbar = [('sd'), ('se'), ('ci')]

    trails_data = combine_trail_data(num_trails=num_trails, num_generation=num_generation)
    for x, y in vis_pair:
        print(trails_data)
    

load_and_vis(num_trails=num_trails, num_generation=num_generation)

In [ ]:
trails_data = combine_trail_data(num_trails=num_trails, num_generation=num_generation)

In [ ]:
trails_data.groupby(['n_gen'])['ikflow_ang'].describe()

In [ ]:
trails_data.describe()

In [ ]:
trails_data.query('n_evals==100200')['ang_errs_min'].describe()

In [ ]:
traj_dir, target_poses = get_target_poses(robot=panda)

In [ ]:
# -> unrefined solutions
solutions, l2_err, _, _, _, _ = ik_solver.solve_n_poses(
    target_poses, 
    refine_solutions=False, 
    return_detailed=True)

iksols = solutions.detach().cpu().numpy()
panda.plot(qs=iksols)

In [ ]:
# Generate iksols
iksols = np.zeros((len(target_poses), num_solutions, panda.dof))

for i, target_pose in enumerate(target_poses):
    solutions = ik_solver.solve(
        target_pose, 
        num_solutions, 
        refine_solutions=False, 
        return_detailed=False)
    iksols[i] = solutions.detach().cpu().numpy()
    
problem = ProblemWrapper(
    iksols=iksols, 
    target_poses=target_poses)

algorithm = BRKGA(
    n_elites=200,
    n_offsprings=700,
    n_mutants=300,
    bias=0.7)

res = minimize(
    problem,
    algorithm,
    ("n_gen", 50),
    seed=1,
    verbose=False,
    save_history=True,
    eliminate_duplicates=MyElementwiseDuplicateElimination())



In [ ]:
qs = res.problem.iksols(res.problem.pheno(res.X))
panda.plot(qs=qs)